In [1]:
import pandas as pd

## **Importar datos**

In [ ]:
# Cargar los archivos CSV desde la carpeta 'data/raw'
df_sales = pd.read_csv('../data/raw/sales.csv', dtype={10: str, 12: str})
df_product_hierarchy = pd.read_csv('../data/raw/product_hierarchy.csv')
df_store_cities = pd.read_csv('../data/raw/store_cities.csv')

In [69]:
df_sales.head()

,product_id,store_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_bin_2,promo_discount_2,promo_discount_type_2
0,P0001,S0002,2017-01-02,0.0,0.00,8.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
1,P0001,S0012,2017-01-02,1.0,5.30,0.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
2,P0001,S0013,2017-01-02,2.0,10.59,0.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
3,P0001,S0023,2017-01-02,0.0,0.00,6.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
4,P0001,S0025,2017-01-02,0.0,0.00,1.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN


In [70]:
df_product_hierarchy.head()

,product_id,product_length,product_depth,product_width,cluster_id,hierarchy1_id,hierarchy2_id,hierarchy3_id,hierarchy4_id,hierarchy5_id
0,P0000,5.0,20.0,12.0,NaN,H00,H0004,H000401,H00040105,H0004010534
1,P0001,13.5,22.0,20.0,cluster_5,H01,H0105,H010501,H01050100,H0105010006
2,P0002,22.0,40.0,22.0,cluster_0,H03,H0315,H031508,H03150800,H0315080028
3,P0004,2.0,13.0,4.0,cluster_3,H03,H0314,H031405,H03140500,H0314050003
4,P0005,16.0,30.0,16.0,cluster_9,H03,H0312,H031211,H03121109,H0312110917


In [71]:
df_store_cities.head()

,store_id,storetype_id,store_size,city_id
0,S0091,ST04,19,C013
1,S0012,ST04,28,C005
2,S0045,ST04,17,C008
3,S0032,ST03,14,C019
4,S0027,ST04,24,C022


## **Exploración de datos**

In [67]:
total_records = len(df_sales)
print(f"Total de registros en el dataset: {total_records}")

Total de registros en el dataset: 19454838


In [65]:
total_decimal_sales = len(df_sales[df_sales['sales'] % 1 != 0])
print(f"Total de registros con decimales en 'sales': {total_decimal_sales}")

Total de registros con decimales en 'sales': 1218907


In [64]:
# Filtrar las filas con valores decimales en 'sales'
decimal_sales = df_sales[df_sales['sales'] % 1 != 0]

# Obtener los product_id únicos con este problema
products_with_decimal_sales = decimal_sales['product_id'].unique()

# Convertir a un DataFrame para un análisis más fácil
products_with_decimal_sales_df = pd.DataFrame(products_with_decimal_sales, columns=['product_id'])

# Agregar conteo de registros por product_id
product_decimal_count = decimal_sales['product_id'].value_counts().reset_index()
product_decimal_count.columns = ['product_id', 'count']

# Mostrar los primeros valores
print(product_decimal_count.head(100))

# Número total de productos con ventas decimales
print(f"Número total de productos con ventas decimales: {len(products_with_decimal_sales)}")


   product_id  count
0       P0413  13278
1       P0663   8185
2       P0125   8173
3       P0131   8171
4       P0333   8156
..        ...    ...
95      P0477   4589
96      P0054   4583
97      P0039   4554
98      P0087   4545
99      P0217   4517

[100 rows x 2 columns]
Número total de productos con ventas decimales: 605


In [59]:
decimal_sales = df_sales[df_sales['sales'] % 1 != 0]
decimal_sales.head()

,product_id,store_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_bin_2,promo_discount_2,promo_discount_type_2
5335,P0298,S0123,2017-01-02,0.530,3.68,6.640,7.49,PR14,NaN,PR03,NaN,NaN,NaN
5336,P0298,S0139,2017-01-02,1.414,9.81,28.428,7.49,PR14,NaN,PR03,NaN,NaN,NaN
5469,P0316,S0062,2017-01-02,2.290,127.00,0.000,59.90,PR14,NaN,PR03,NaN,NaN,NaN
5470,P0316,S0085,2017-01-02,2.580,143.09,0.000,59.90,PR14,NaN,PR03,NaN,NaN,NaN
7270,P0413,S0001,2017-01-02,1.695,3.12,4.415,1.99,PR03,verylow,PR03,NaN,NaN,NaN


## **Transformación de datos**

In [5]:
df_sales['date'] = pd.to_datetime(df_sales['date'])

## **Productos más populares cada mes durante el año 2018**

In [23]:
#Filtrar año 2018 y agregar columnas para la identificación del mes
df_2018 = df_sales[df_sales['date'].dt.year == 2018].copy()
df_2018['month'] = df_2018['date'].dt.month_name()
df_2018['month_num'] = df_2018['date'].dt.month

In [34]:
#Agregar ventas por producto y mes
product_sales = (
    df_2018.groupby(['month_num','month', 'product_id'])['sales']
    .sum()
    .reset_index()
    .sort_values(['month_num', 'sales'], ascending=[True, False])
)

In [37]:
#Mostar los 5 primeros productos con más ventas por mes
top_5_products_by_month = (
    product_sales.groupby('month')
    .head(5)
    .reset_index(drop=True)[['month', 'product_id', 'sales']]
)

In [68]:
top_5_products_by_month.head(10)

,month,product_id,sales
0,January,P0438,21326.0
1,January,P0103,19046.0
2,January,P0364,7984.0
3,January,P0051,7754.0
4,January,P0590,5692.0
5,February,P0438,21485.0
6,February,P0103,16474.0
7,February,P0051,7555.0
8,February,P0388,6212.0
9,February,P0590,5952.0


## **En cuántas tiendas se implementaron promociones del tipo 1 en el canal 1 durante el año 2019**

In [7]:
#Contar los valores únicos de store_id donde promo_type_1 tiene un valor válido
unique_store_ids_2019 = df_sales[(df_sales['promo_type_1'].notna()) 
& (df_sales['date'].dt.year == 2019)]['store_id'].nunique()
print(f"Se eimplementó la promoción del tipo 1 en el canal 1 en {unique_store_ids_2019} tiendas")

Se eimplementó la promoción del tipo 1 en el canal 1 en 144 tiendas


## **Cuál fue la ciudad que tuvo el mayor volumen de ventas en general durante todo el periodo de 2017 a 2019**

In [90]:
#Seleccionar columnas a unir
columns_sales = ['store_id', 'date', 'sales', 'price', 'revenue']
columns_store_cities = ['store_id', 'city_id']

#Unir columnas por store_id
df_merged_sales_store_cities = pd.merge(
    df_sales[columns_sales],
    df_store_cities[columns_store_cities],
    on='store_id',
    how='left'
)

In [91]:
df_merged_sales_store_cities.head()

,store_id,date,sales,price,revenue,city_id
0,S0002,2017-01-02,0.0,6.25,0.00,C007
1,S0012,2017-01-02,1.0,6.25,5.30,C005
2,S0013,2017-01-02,2.0,6.25,10.59,C026
3,S0023,2017-01-02,0.0,6.25,0.00,C008
4,S0025,2017-01-02,0.0,6.25,0.00,C024


In [100]:
#Mostrar la tienda con las mayores ventas
top_city = df_merged_sales_store_cities.groupby('city_id')['sales'].sum().idxmax()
print(f"Ciudad con las mayores ventas 2017 a 2019: {top_city}")

Ciudad con las mayores ventas 2017 a 2019: C014


## **Cuál fue el promedio de stock disponible en todas las tiendas durante el año 2017**

In [8]:
df_2017 = df_sales[df_sales['date'].dt.year == 2017].copy()

In [17]:
# Calcular el promedio de por tienda
promedio_por_tienda = df_2017.groupby('store_id')['stock'].mean()

# Calcular el promedio general
promedio_stock_total = round(promedio_por_tienda.mean(),2)

print(f"Promedio de stock disponible en todas las tiendas en 2017: {promedio_stock_total}")

Promedio de stock disponible en todas las tiendas en 2017: 15.85


## **Cuántas veces se aplicaron descuentos en el canal 2 durante el mes de diciembre de 2018**

In [25]:
#Filtrar rango de fecha a usar
df_diciembre_2018 = df_2018[df_2018['month_num'] == 12]

In [56]:
# Contar registros donde tanto promo_type_2 como promo_discount_2 no son nulos
conteo_con_descuento = df_diciembre_2018[(df_diciembre_2018['promo_type_2'].notna()) & (df_diciembre_2018['promo_discount_2'].notna())].shape[0]

# Imprimir el resultado
print(f"Para diciembre 2018 en el canal 2 se aplicaron {conteo_con_descuento} descuentos")

Para diciembre 2018 en el canal 2 se aplicaron 0 descuentos


In [58]:
df_sales.head()

,product_id,store_id,date,sales,revenue,stock,price,promo_type_1,promo_bin_1,promo_type_2,promo_bin_2,promo_discount_2,promo_discount_type_2
0,P0001,S0002,2017-01-02,0.0,0.00,8.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
1,P0001,S0012,2017-01-02,1.0,5.30,0.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
2,P0001,S0013,2017-01-02,2.0,10.59,0.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
3,P0001,S0023,2017-01-02,0.0,0.00,6.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN
4,P0001,S0025,2017-01-02,0.0,0.00,1.0,6.25,PR14,NaN,PR03,NaN,NaN,NaN


## **Cuál fue el producto con el precio de venta más alto en cada tienda durante el tercer trimestre de 2019**

In [ ]:
# Filtrar los datos para el tercer trimestre de 2019.
df_2019_q3 = df_sales[(df_sales['date'].dt.year == 2019) & (df_sales['date'].dt.month.isin([7, 8, 9]))]

# Producto con el precio de venta más alto por tienda
producto_mas_caros_por_tienda = df_2019_q3.loc[df_2019_q3.groupby('store_id')['price'].idxmax()]

# Seleccionar las columnas a usar
producto_mas_caros_por_tienda = producto_mas_caros_por_tienda[['store_id', 'product_id', 'price']]

# Ordenar por tienda y precio de mayor a menor
producto_mas_caros_por_tienda = producto_mas_caros_por_tienda.sort_values(by=['store_id', 'price'], ascending=[True, False])

In [65]:
producto_mas_caros_por_tienda.head(10)

,store_id,product_id,price
16204268,S0001,P0498,299.90
16991302,S0002,P0708,1599.00
15771337,S0003,P0498,299.90
15771338,S0004,P0498,299.90
15771339,S0005,P0498,299.90
16127071,S0006,P0680,139.95
15768426,S0007,P0391,179.90
15771340,S0008,P0498,299.90
16127074,S0009,P0680,139.95
16127075,S0010,P0680,139.95


## **Cuántas tiendas experimentaron un aumento de ventas del 20% o más en comparación con el año anterior**

In [79]:
# Filtrar los datos para los años 2017, 2018 y 2019
df_2017 = df_sales[df_sales['date'].dt.year == 2017]
df_2018 = df_sales[df_sales['date'].dt.year == 2018]
df_2019 = df_sales[df_sales['date'].dt.year == 2019]

# Agrupar por tienda y calcular las ventas totales por tienda para cada año
ventas_2017 = df_2017.groupby('store_id')['sales'].sum().reset_index(name='ventas_2017')
ventas_2018 = df_2018.groupby('store_id')['sales'].sum().reset_index(name='ventas_2018')
ventas_2019 = df_2019.groupby('store_id')['sales'].sum().reset_index(name='ventas_2019')

In [80]:
# Hacer un left join entre los DataFrames de ventas por año
ventas_comparadas = pd.merge(ventas_2017, ventas_2018, on='store_id', how='left')
ventas_comparadas = pd.merge(ventas_comparadas, ventas_2019, on='store_id', how='left')

In [81]:
# Calcular el porcentaje de cambio de las ventas entre 2017 y 2018, y 2018 y 2019
ventas_comparadas['cambio_2017_2018'] = (ventas_comparadas['ventas_2018'] - ventas_comparadas['ventas_2017']) / ventas_comparadas['ventas_2017'] * 100
ventas_comparadas['cambio_2018_2019'] = (ventas_comparadas['ventas_2019'] - ventas_comparadas['ventas_2018']) / ventas_comparadas['ventas_2018'] * 100

In [82]:
ventas_comparadas.head()

,store_id,ventas_2017,ventas_2018,ventas_2019,cambio_2017_2018,cambio_2018_2019
0,S0001,45905.874,48685.417,43315.438,6.054874,-11.029954
1,S0002,37411.430,37387.370,30391.990,-0.064312,-18.710543
2,S0003,12932.520,13269.110,10030.540,2.602664,-24.406837
3,S0004,14135.300,12896.315,10567.000,-8.765184,-18.061865
4,S0006,3673.000,4091.000,3301.000,11.380343,-19.310682


In [83]:
# Filtrar las tiendas con un aumento de ventas del 20% o más entre 2017 y 2018, y entre 2018 y 2019
tiendas_aumento_20_2017_2018 = ventas_comparadas[ventas_comparadas['cambio_2017_2018'] >= 20]
tiendas_aumento_20_2018_2019 = ventas_comparadas[ventas_comparadas['cambio_2018_2019'] >= 20]

In [84]:
tiendas_aumento_20_2017_2018.head()

,store_id,ventas_2017,ventas_2018,ventas_2019,cambio_2017_2018,cambio_2018_2019
5,S0008,14768.985,20940.145,21065.550,41.784591,0.598874
6,S0009,5862.000,7342.100,6220.000,25.249062,-15.283093
14,S0017,8261.575,10566.000,8169.000,27.893289,-22.685974
15,S0018,19001.326,22901.791,16448.596,20.527331,-28.177687
31,S0034,4147.000,6751.000,6319.000,62.792380,-6.399052


In [85]:
tiendas_aumento_20_2018_2019.head()

,store_id,ventas_2017,ventas_2018,ventas_2019,cambio_2017_2018,cambio_2018_2019
117,S0134,3163.0,4027.0,5275.0,27.315839,30.990812


In [86]:
# Contar cuántas tiendas tienen un aumento de ventas del 20% o más en cada comparación
numero_tiendas_aumento_20_2017_2018 = tiendas_aumento_20_2017_2018.shape[0]
numero_tiendas_aumento_20_2018_2019 = tiendas_aumento_20_2018_2019.shape[0]

# Imprimir el resultado
print(f"El número de tiendas que experimentaron un aumento de ventas del 20% o más entre 2017 y 2018 es: {numero_tiendas_aumento_20_2017_2018}")
print(f"El número de tiendas que experimentaron un aumento de ventas del 20% o más entre 2018 y 2019 es: {numero_tiendas_aumento_20_2018_2019}")


El número de tiendas que experimentaron un aumento de ventas del 20% o más entre 2017 y 2018 es: 24
El número de tiendas que experimentaron un aumento de ventas del 20% o más entre 2018 y 2019 es: 1


## **Cuál fue el producto que experimentó la mayor variación de precio a lo largo del periodo de 2017 a 2019**

In [90]:
# Agrupar por product_id y calcular el precio máximo y mínimo para cada producto
precio_maximo = df_sales.groupby('product_id')['price'].max()
precio_minimo = df_sales.groupby('product_id')['price'].min()

In [95]:
# Calcular la variación de precio (diferencia entre el precio máximo y mínimo)
variacion_precio = precio_maximo - precio_minimo

#Extraer el producto con la mayor variacion
producto_mayor_variacion = variacion_precio.idxmax()

In [97]:
# Obtener el precio máximo y mínimo para ese producto
precio_max = precio_maximo[producto_mayor_variacion]
precio_min = precio_minimo[producto_mayor_variacion]

# Mostrar el resultado
print(f"Producto con la mayor variación de precio: {producto_mayor_variacion}")
print(f"Precio máximo: {precio_max}")
print(f"Precio mínimo: {precio_min}")
print(f"Variación de precio: {precio_max - precio_min}")

Producto con la mayor variación de precio: P0632
Precio máximo: 849.9
Precio mínimo: 549.9
Variación de precio: 300.0


## **En qué mes y año se registró la mayor cantidad de ingresos generados en todas las tiendas**

In [ ]:
# Crear una nueva columna para el mes y año
df_sales['year_month'] = df_sales['date'].dt.to_period('M')

# Agrupar por la columna year_month y sumar los ingresos
ingresos_por_mes = df_sales.groupby('year_month')['revenue'].sum()

In [ ]:
# Encontrar el mes y año con los mayores ingresos
mes_max_ingresos = ingresos_por_mes.idxmax()
max_ingresos = ingresos_por_mes.max()

In [ ]:
# Mostrar el resultado
print(f"El mes y año con mayor cantidad de ingresos fue {mes_max_ingresos} con un total de ingresos de {max_ingresos}.")

## **Cuál fue el tamaño promedio de las tiendas en cada ciudad durante el año 2018**

In [124]:
# Store_id únicos en 2018
store_ids_unicos_2018 = df_2018['store_id'].drop_duplicates()

In [125]:
# Filtrar df_store_cities para obtener solo las tiendas que están en store_ids_unicos_2018
df_filtered_stores = df_store_cities[df_store_cities['store_id'].isin(store_ids_unicos_2018)]

# Agrupar por ciudad y calcular el tamaño promedio de las tiendas
tamaño_promedio_ciudad = df_filtered_stores.groupby('city_id')['store_size'].mean().reset_index()

In [126]:
# Renombrar la columna
tamaño_promedio_ciudad.rename(columns={'store_size': 'tamaño_promedio'}, inplace=True)

# Redondear la columna
tamaño_promedio_ciudad['tamaño_promedio'] = tamaño_promedio_ciudad['tamaño_promedio'].round(2)

In [127]:
tamaño_promedio_ciudad.head(10)

,city_id,tamaño_promedio
0,C001,20.00
1,C002,47.00
2,C003,13.00
3,C004,63.00
4,C005,19.00
5,C006,29.67
6,C007,39.00
7,C008,23.67
8,C009,20.00
9,C010,23.00
